In [9]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
from dotenv import load_dotenv
import os

In [10]:
load_dotenv()

BASE_URL = "https://api.themoviedb.org/3"
API_KEY = os.getenv("TMDB_API_KEY")

In [4]:
df = pd.read_csv('./data/wikidata.csv')
df

,item,itemLabel,enLabel,pubYear,imdbID,tmdbID,type,genres,cast,directors,screenwriters,runtimes
0,http://www.wikidata.org/entity/Q110393291,승부,The Match,2025,tt13498824,760497,movie,"드라마 영화, 전기 영화","현봉식, 김강훈, 고창석, 문정희, 유아인, 이병헌",김형주,"윤종빈, 김형주",115
1,http://www.wikidata.org/entity/Q110585119,하이파이브,Hi-Five,2025,tt14044924,793058,movie,액션 영화,"이재인, 안재홍, 라미란, 김희원, 오정세, 신구, 박진영, 유아인",강형철,NaN,119
2,http://www.wikidata.org/entity/Q112977213,미키 17,Mickey 17,2025,tt12299608,696506,movie,"드라마 영화, 코미디 영화, 액션 영화, SF 영화, 모험 영화","브론윈 제임스, 나오미 애키, 패치 퍼랜, 캐머런 브리턴, 스티브 박, 토머스 터구...",봉준호,봉준호,137
3,http://www.wikidata.org/entity/Q120922062,보통의 가족,A Normal Family,2025,tt28488187,973628,movie,"드라마 영화, 스릴러 영화",NaN,허진호,NaN,109
4,http://www.wikidata.org/entity/Q125971387,부고니아,NaN,2025,tt12300742,701387,movie,"코미디 영화, SF 영화","제시 플레먼스, 얼리시아 실버스톤, 엠마 스톤",요르고스 란티모스,NaN,118
...,...,...,...,...,...,...,...,...,...,...,...,...
951,http://www.wikidata.org/entity/Q50322115,Naneun Seonmuda,I Am Sun Mu,2015,tt4722674,383353,movie,다큐멘터리 영화,NaN,NaN,NaN,NaN
952,http://www.wikidata.org/entity/Q55734574,중급불어,NaN,2015,tt5258552,449802,movie,드라마 영화,NaN,NaN,NaN,NaN
953,http://www.wikidata.org/entity/Q65241097,다방의 푸른 꿈,Try To Remember,2015,tt7520904,601068,movie,독립 영화,NaN,NaN,NaN,NaN
954,http://www.wikidata.org/entity/Q65300188,퀴어영화 나비: 어른들의 일,Butterfly: The Adult World,2015,tt5358748,510589,movie,성 소수자 연관 장르,NaN,NaN,NaN,NaN


In [5]:
# 'tmdbID' 행(열)에 있는 NaN(결측치)의 개수를 확인
nan_count = df['tmdbID'].isna().sum()

print(f"tmdbID 행의 nan 개수: {nan_count}")

tmdbID 행의 nan 개수: 0


In [12]:
import pandas as pd
import requests
import json
import os
import time
from tqdm import tqdm
from dotenv import load_dotenv

# --- 1. 환경 설정 ---

# .env 파일 로드
load_dotenv()

# OS 환경 변수에서 API 키 가져오기
API_KEY = os.getenv("TMDB_API_KEY")

# API 키가 없는 경우 오류 발생
if not API_KEY:
    raise ValueError("TMDB_API_KEY가 .env 파일에 설정되지 않았습니다. (예: TMDB_API_KEY=your_key_here)")

# TMDB API 기본 URL
BASE_URL = "https://api.themoviedb.org/3"

# API 요청 시 사용할 파라미터 
# (언어 설정이 필요한 경우)
params_ko = {
    "api_key": API_KEY,
    "language": "ko-KR"
}
# (언어 설정이 필요 없거나 지원되지 않는 경우)
params_plain = {
    "api_key": API_KEY
}

# --- 2. 1단계 CSV 파일 로드 ---

INPUT_CSV = "./data/wikidata.csv"
OUTPUT_JSONL = "output/tmdb_data.jsonl"

try:
    df = pd.read_csv(INPUT_CSV)
    # 'type' 열이 없는 구버전 쿼리일 경우를 대비 (필수는 아님)
    if 'type' not in df.columns:
        print("경고: seed_list.csv에 'type' 열이 없습니다. 이전 쿼리를 수정하는 것을 권장합니다.")
        # 임시방편: P4947 (영화 ID)가 있으면 movie, 아니면 tv로 추정
        # 이 부분은 SPARQL 쿼리를 수정하는 것이 가장 좋습니다.
        # 여기서는 'type'이 있다고 가정하고 진행합니다.

except FileNotFoundError:
    print(f"오류: 1단계 파일인 '{INPUT_CSV}'를 찾을 수 없습니다.")
    print("스크립트를 종료합니다.")
    exit()
except Exception as e:
    print(f"파일 로드 중 오류 발생: {e}")
    exit()


# --- 3. 데이터 보강 및 JSONL 저장 ---

print(f"'{INPUT_CSV}' 파일에서 데이터 보강을 시작합니다. 결과는 '{OUTPUT_JSONL}'에 저장됩니다.")

# JSONL 파일로 결과 저장 (이어쓰기 'a' 모드)
with open(OUTPUT_JSONL, "a", encoding="utf-8") as output_file:
    
    # tqdm으로 진행률 표시
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing items"):
        
        # 1단계 CSV에서 기본 정보 추출
        try:
            tmdb_id = int(row['tmdbID']) # 정수형으로 변환
            item_type = row['type'].strip() # 'movie' or 'tv', 공백 제거
            qid = row.get('item', '')
            imdb_id = row.get('imdbID', '')
        except Exception as e:
            print(f"Skipping row {index}: 필수 데이터(tmdbID, type) 로드 실패. {e}")
            continue

        # API 호출을 위한 기본 경로 설정
        base_path = f"{BASE_URL}/{item_type}/{tmdb_id}"
        
        # 이 작품의 모든 데이터를 담을 딕셔너리
        enriched_data = {
            "qid": qid,
            "imdb_id": imdb_id,
            "tmdb_id": tmdb_id,
            "type": item_type,
            "title_ko": row.get('itemLabel'),
            "title_en": row.get('enLabel'),
            "year": int(row.get('pubYear', 0))
        }

        try:
            # --- 5.1: /details (핵심 정보) ---
            details_resp_json = requests.get(base_path, params=params_ko).json()
            
            enriched_data['overview'] = details_resp_json.get('overview', '')
            enriched_data['genres'] = [g['name'] for g in details_resp_json.get('genres', [])]
            enriched_data['poster_path'] = details_resp_json.get('poster_path', '')
            enriched_data['backdrop_path'] = details_resp_json.get('backdrop_path', '')
            
            # 런타임은 movie/tv 구조가 다름
            if item_type == 'movie':
                enriched_data['runtime_min'] = details_resp_json.get('runtime', 0)
            elif item_type == 'tv':
                # TV는 에피소드 평균 런타임
                rt_list = details_resp_json.get('episode_run_time', [])
                enriched_data['runtime_min'] = rt_list[0] if rt_list else 0

            # --- 5.2: /credits (출연/제작진) ---
            credits_resp_json = requests.get(f"{base_path}/credits", params=params_ko).json()
            
            # 주요 출연진 10명
            enriched_data['cast'] = [c['name'] for c in credits_resp_json.get('cast', [])[:10]]
            
            # 감독 및 작가
            crew = credits_resp_json.get('crew', [])
            enriched_data['directors'] = [c['name'] for c in crew if c.get('job') == 'Director']
            enriched_data['writers'] = [c['name'] for c in crew if c.get('job') in ('Writer', 'Screenplay')]

            # --- 5.3: /watch/providers (OTT 정보) ---
            providers_resp_json = requests.get(f"{base_path}/watch/providers", params=params_plain).json()
            
            # 'KR' (한국) 정보만 추출
            kr_providers = providers_resp_json.get('results', {}).get('KR', {})
            
            enriched_data['ott_streaming_kr'] = [p['provider_name'] for p in kr_providers.get('flatrate', [])]
            enriched_data['ott_rent_kr'] = [p['provider_name'] for p in kr_providers.get('rent', [])]
            enriched_data['ott_buy_kr'] = [p['provider_name'] for p in kr_providers.get('buy', [])]

            # --- 5.4: /keywords (RAG 핵심 키워드) ---
            keywords_resp_json = requests.get(f"{base_path}/keywords", params=params_plain).json()
            
            # movie/tv 구조가 다름
            keywords_list = []
            if item_type == 'movie':
                keywords_list = keywords_resp_json.get('keywords', [])
            elif item_type == 'tv':
                keywords_list = keywords_resp_json.get('results', [])
                
            enriched_data['keywords'] = [k['name'] for k in keywords_list]

            # --- 5.5: 연령 등급 (국가별) ---
            rating_kr = "" # 기본값
            
            if item_type == 'movie':
                # 영화: /release_dates
                ratings_resp_json = requests.get(f"{base_path}/release_dates", params=params_plain).json()
                all_ratings = ratings_resp_json.get('results', [])
                for r in all_ratings:
                    if r.get('iso_3166_1') == 'KR':
                        # 한국 데이터 중 'certification'이 있는 첫 번째 항목을 사용
                        for release in r.get('release_dates', []):
                            cert = release.get('certification')
                            if cert:
                                rating_kr = cert
                                break
                        if rating_kr: # 찾았으면 종료
                            break
                            
            elif item_type == 'tv':
                # TV: /content_ratings
                ratings_resp_json = requests.get(f"{base_path}/content_ratings", params=params_plain).json()
                all_ratings = ratings_resp_json.get('results', [])
                for r in all_ratings:
                    if r.get('iso_3166_1') == 'KR':
                        rating_kr = r.get('rating', '')
                        break # 찾았으면 종료
            
            enriched_data['rating_kr'] = rating_kr

            # --- 6. 파일에 쓰기 ---
            # 완성된 JSON 객체를 한 줄로 변환하여 파일에 추가
            output_file.write(json.dumps(enriched_data, ensure_ascii=False) + "\n")
            
            # API Rate Limiting 방지를 위해 잠시 대기
            time.sleep(0.05) # 0.1초 (초당 10회) ~ 0.05초 (초당 20회) 권장

        except requests.exceptions.RequestException as e:
            print(f"Error (Request) processing {tmdb_id}: {e}")
        except Exception as e:
            # JSON 파싱 오류, 키 오류 등
            print(f"Error (Processing) processing {tmdb_id}: {e}")
            # 오류가 발생해도 다음 항목으로 계속 진행

print("--- 데이터 보강 완료 ---")
print(f"최종 데이터가 '{OUTPUT_JSONL}' 파일에 저장되었습니다.")

'./data/wikidata.csv' 파일에서 데이터 보강을 시작합니다. 결과는 'output/tmdb_data.jsonl'에 저장됩니다.


Processing items:  18%|█▊        | 169/956 [08:40<1:58:04,  9.00s/it]

Error (Request) processing 743814: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing items:  64%|██████▎   | 608/956 [38:17<21:54,  3.78s/it]  


KeyboardInterrupt: 

In [13]:
import pandas as pd
import requests
import json
import os
import time
from tqdm import tqdm
from dotenv import load_dotenv

# --- 1. 환경 설정 ---
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")

if not API_KEY:
    raise ValueError("TMDB_API_KEY가 .env 파일에 설정되지 않았습니다.")

BASE_URL = "https://api.themoviedb.org/3"
params_ko = {"api_key": API_KEY, "language": "ko-KR"}
params_plain = {"api_key": API_KEY}

# --- 2. 입/출력 파일 정의 ---
# (사용자님이 말씀해주신 경로로 수정)
INPUT_CSV = "./data/wikidata.csv" 
OUTPUT_JSONL = "./output/tmdb_data.jsonl"

# --- [수정 1] '이어하기' 로직: 이미 처리된 ID 목록을 로드 ---
processed_ids = set()
try:
    # 출력 파일이 이미 존재하면, ID를 읽어와서 set에 저장
    with open(OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data = json.loads(line)
                if 'tmdb_id' in data:
                    processed_ids.add(data['tmdb_id']) # int형으로 저장
            except json.JSONDecodeError:
                # JSON 형식이 깨진 줄은 무시
                continue
    if processed_ids:
        print(f"총 {len(processed_ids)}개의 기처리된 항목을 발견했습니다. 이어서 작업을 시작합니다.")
except FileNotFoundError:
    # 출력 파일이 없으면 처음부터 시작
    print(f"'{OUTPUT_JSONL}' 파일을 찾을 수 없습니다. 처음부터 시작합니다.")
# --- [수정 1] 끝 ---


# --- 3. 1단계 CSV 파일 로드 ---
try:
    df = pd.read_csv(INPUT_CSV)
except FileNotFoundError:
    print(f"오류: 1단계 파일인 '{INPUT_CSV}'를 찾을 수 없습니다.")
    exit()

# --- 4. 데이터 보강 및 JSONL 저장 ---
print(f"'{INPUT_CSV}' 파일에서 데이터 보강을 시작합니다. 결과는 '{OUTPUT_JSONL}'에 저장됩니다.")

with open(OUTPUT_JSONL, "a", encoding="utf-8") as output_file:
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing items"):
        
        try:
            tmdb_id = int(row['tmdbID'])
            item_type = row['type'].strip()
            qid = row.get('item', '')
            imdb_id = row.get('imdbID', '')
        except Exception as e:
            print(f"Skipping row {index}: 필수 데이터(tmdbID, type) 로드 실패. {e}")
            continue

        # --- [수정 2] '이어하기' 로직: 이미 처리했는지 확인 ---
        if tmdb_id in processed_ids:
            continue # 이미 set에 ID가 존재하면 API 호출을 모두 건너뜁니다.
        # --- [수정 2] 끝 ---

        # (기본 로직은 동일)
        base_path = f"{BASE_URL}/{item_type}/{tmdb_id}"
        enriched_data = {
            "qid": qid, "imdb_id": imdb_id, "tmdb_id": tmdb_id, "type": item_type,
            "title_ko": row.get('itemLabel'), "title_en": row.get('enLabel'),
            "year": int(row.get('pubYear', 0))
        }

        try:
            # --- [수정 3] 모든 API 호출에 'timeout=10' (10초) 추가 ---
            
            # 5.1: /details
            details_resp_json = requests.get(base_path, params=params_ko, timeout=10).json()
            enriched_data['overview'] = details_resp_json.get('overview', '')
            enriched_data['genres'] = [g['name'] for g in details_resp_json.get('genres', [])]
            # (이하 details 추출 로직 생략...)

            # 5.2: /credits
            credits_resp_json = requests.get(f"{base_path}/credits", params=params_ko, timeout=10).json()
            enriched_data['cast'] = [c['name'] for c in credits_resp_json.get('cast', [])[:10]]
            crew = credits_resp_json.get('crew', [])
            enriched_data['directors'] = [c['name'] for c in crew if c.get('job') == 'Director']
            enriched_data['writers'] = [c['name'] for c in crew if c.get('job') in ('Writer', 'Screenplay')]

            # 5.3: /watch/providers
            providers_resp_json = requests.get(f"{base_path}/watch/providers", params=params_plain, timeout=10).json()
            kr_providers = providers_resp_json.get('results', {}).get('KR', {})
            enriched_data['ott_streaming_kr'] = [p['provider_name'] for p in kr_providers.get('flatrate', [])]
            # (이하 providers 추출 로직 생략...)

            # 5.4: /keywords
            keywords_resp_json = requests.get(f"{base_path}/keywords", params=params_plain, timeout=10).json()
            keywords_list = keywords_resp_json.get('keywords', []) if item_type == 'movie' else keywords_resp_json.get('results', [])
            enriched_data['keywords'] = [k['name'] for k in keywords_list]

            # 5.5: 연령 등급
            rating_kr = ""
            if item_type == 'movie':
                ratings_resp_json = requests.get(f"{base_path}/release_dates", params=params_plain, timeout=10).json()
                # (이하 rating 추출 로직 생략...)
            elif item_type == 'tv':
                ratings_resp_json = requests.get(f"{base_path}/content_ratings", params=params_plain, timeout=10).json()
                # (이하 rating 추출 로직 생략...)
            
            enriched_data['rating_kr'] = rating_kr # (추출 로직은 이전과 동일)

            # --- 6. 파일에 쓰기 ---
            output_file.write(json.dumps(enriched_data, ensure_ascii=False) + "\n")
            
            # (API Rate Limit 방지 대기)
            time.sleep(0.05) 

        # --- [수정 4] Timeout 전용 오류 처리 추가 ---
        except requests.exceptions.Timeout:
            # 10초 내에 응답이 없으면 이 예외가 발생
            print(f"Error (Timeout) processing {tmdb_id}: 서버가 10초 내에 응답하지 않았습니다. 건너뜁니다.")
        
        except requests.exceptions.RequestException as e:
            print(f"Error (Request) processing {tmdb_id}: {e}")
        except Exception as e:
            print(f"Error (Processing) processing {tmdb_id}: {e}")
            
print("--- 데이터 보강 완료 ---")

총 540개의 기처리된 항목을 발견했습니다. 이어서 작업을 시작합니다.
'./data/wikidata.csv' 파일에서 데이터 보강을 시작합니다. 결과는 './output/tmdb_data.jsonl'에 저장됩니다.


Processing items: 100%|██████████| 956/956 [16:59<00:00,  1.07s/it] 

--- 데이터 보강 완료 ---


In [2]:
import json
import os
import requests
import time
from tqdm import tqdm
from requests.utils import quote # URL 인코딩을 위한 import

# --- 1. 입/출력 파일 설정 ---
INPUT_JSONL = "./output/tmdb_data.jsonl"
OUTPUT_JSONL = "./output/rag_data.jsonl"
WIKI_API_ENDPOINT = "https://ko.wikipedia.org/api/rest_v1/page/summary/"

# --- 2. '이어하기' 로직: 이미 처리된 ID 목록 로드 ---
processed_ids = set()
try:
    # 출력 파일이 이미 존재하면, ID를 읽어와서 set에 저장
    with open(OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data = json.loads(line)
                if 'tmdb_id' in data:
                    processed_ids.add(data['tmdb_id']) # tmdb_id를 기준으로
            except json.JSONDecodeError:
                continue
    if processed_ids:
        print(f"총 {len(processed_ids)}개의 기처리된 항목(RAG 텍스트)을 발견했습니다. 이어서 작업을 시작합니다.")
except FileNotFoundError:
    print(f"'{OUTPUT_JSONL}' 파일을 찾을 수 없습니다. 처음부터 시작합니다.")


# --- 3. Wikipedia 요약 가져오기 함수 ---
def get_wikipedia_summary(title_ko):
    """
    한국어 Wikipedia API를 호출하여 작품 요약(extract)을 가져옵니다.
    """
    # API 요청 시 'User-Agent' 헤더를 설정하는 것이 좋습니다.
    headers = {
        'User-Agent': 'OTTBotDataCollector/1.0 (test@example.com)'
    }
    
    # 제목을 URL에 맞게 인코딩 (예: "승부" -> "%EC%8A%B9%EB%B6%80")
    encoded_title = quote(title_ko)
    
    try:
        response = requests.get(WIKI_API_ENDPOINT + encoded_title, headers=headers, timeout=5)
        
        # 404: 페이지 없음
        if response.status_code == 404:
            # print(f"'{title_ko}'의 위키피디아 문서를 찾을 수 없습니다 (404).")
            return ""
        
        # 200: 성공
        elif response.status_code == 200:
            data = response.json()
            # 'type'이 'disambiguation'(동음이의어) 페이지인 경우 요약이 없음
            if data.get('type') == 'disambiguation':
                # print(f"'{title_ko}'는 동음이의어 페이지입니다. 건너뜁니다.")
                return ""
            
            # 'extract' 필드(요약) 반환
            return data.get('extract', "")
        
        # 기타 오류
        else:
            # print(f"'{title_ko}' 처리 중 API 오류 발생 (Status: {response.status_code}).")
            return ""
            
    except requests.exceptions.Timeout:
        # print(f"'{title_ko}' 요청 시간 초과.")
        return ""
    except requests.exceptions.RequestException as e:
        # print(f"'{title_ko}' 요청 중 네트워크 오류: {e}")
        return ""


# --- 4. RAG용 텍스트 생성 및 저장 ---

try:
    # 2단계 결과 파일(입력) 열기
    with open(INPUT_JSONL, "r", encoding="utf-8") as infile, \
         open(OUTPUT_JSONL, "a", encoding="utf-8") as outfile: # 3단계 결과 파일(출력) 열기

        # 입력 파일의 전체 라인 수를 세서 tqdm에 적용
        # (파일이 클 경우 이 부분이 시간이 걸릴 수 있음)
        print("입력 파일의 전체 라인 수를 세는 중...")
        infile.seek(0) # 파일 포인터 맨 앞으로
        total_lines = sum(1 for line in infile)
        infile.seek(0) # 다시 맨 앞으로
        print(f"총 {total_lines}개의 항목을 처리합니다.")
        
        # tqdm으로 입력 파일 순회
        for line in tqdm(infile, total=total_lines, desc="Creating RAG text"):
            try:
                data = json.loads(line)
                tmdb_id = data.get('tmdb_id')
                
                # '이어하기' 확인
                if not tmdb_id or tmdb_id in processed_ids:
                    continue

                title_ko = data.get('title_ko', '')
                
                # --- 3.1: Wikipedia 요약 가져오기 ---
                wikipedia_summary = get_wikipedia_summary(title_ko)
                
                # API 부하를 줄이기 위해 잠시 대기
                time.sleep(0.05) 

                # --- 3.2: 텍스트 결합 ---
                # RAG 모델이 잘 이해할 수 있도록 명확한 필드로 텍스트 구성
                text_blob_parts = []
                
                text_blob_parts.append(f"[제목] {title_ko}")
                if data.get('title_en'):
                    text_blob_parts.append(f"[영문 제목] {data.get('title_en')}")
                
                if data.get('overview'):
                    text_blob_parts.append(f"[줄거리] {data.get('overview')}")
                
                # Wikipedia 요약이 TMDB 줄거리와 다를 경우에만 추가 (중복 방지)
                if wikipedia_summary and wikipedia_summary not in data.get('overview', ''):
                    text_blob_parts.append(f"[추가 요약] {wikipedia_summary}")
                    
                if data.get('genres'):
                    text_blob_parts.append(f"[장르] {', '.join(data.get('genres', []))}")
                    
                if data.get('keywords'):
                    text_blob_parts.append(f"[키워드] {', '.join(data.get('keywords', []))}")
                    
                if data.get('cast'):
                    # 출연진은 RAG 성능에 매우 중요 (예: "이병헌 나오는 영화")
                    text_blob_parts.append(f"[주요 출연진] {', '.join(data.get('cast', []))}")
                
                if data.get('directors'):
                    text_blob_parts.append(f"[감독] {', '.join(data.get('directors', []))}")

                # 모든 텍스트를 줄바꿈(\n)으로 연결
                rag_text = "\n".join(text_blob_parts)
                
                # --- 3.3: 새 데이터 객체 생성 및 저장 ---
                # 원본 데이터(data)에 'rag_text' 필드를 추가
                data['rag_text'] = rag_text
                
                # 완성된 객체를 새 JSONL 파일에 저장
                outfile.write(json.dumps(data, ensure_ascii=False) + "\n")
                
                # 처리된 ID 목록에 추가 (메모리상)
                processed_ids.add(tmdb_id)

            except json.JSONDecodeError:
                print("JSON 파싱 오류. 해당 줄을 건너뜁니다.")
                continue
            except Exception as e:
                print(f"처리 중 알 수 없는 오류 발생 (데이터: {line[:50]}...): {e}")
                continue

except FileNotFoundError:
    print(f"오류: 2단계 파일인 '{INPUT_JSONL}'을 찾을 수 없습니다.")
except Exception as e:
    print(f"스크립트 실행 중 치명적인 오류 발생: {e}")

print("--- 3단계 (RAG 텍스트 생성) 완료 ---")
print(f"최종 RAG 데이터가 '{OUTPUT_JSONL}' 파일에 저장되었습니다.")

총 860개의 기처리된 항목(RAG 텍스트)을 발견했습니다. 이어서 작업을 시작합니다.
입력 파일의 전체 라인 수를 세는 중...
총 936개의 항목을 처리합니다.


Creating RAG text: 100%|██████████| 936/936 [00:00<00:00, 95860.44it/s]

--- 3단계 (RAG 텍스트 생성) 완료 ---
최종 RAG 데이터가 './output/rag_data.jsonl' 파일에 저장되었습니다.


In [15]:
import pandas as pd

# 3단계에서 생성된 파일 경로
file_path = './output/rag_data.jsonl'

try:
    # 'lines=True' 옵션이 JSONL 파일을 한 줄씩 읽어 DataFrame으로 만듭니다.
    df = pd.read_json(file_path, lines=True)

    print(f"✅ '{file_path}' 파일을 DataFrame으로 성공적으로 로드했습니다.")
    print(f"Shape: {df.shape} (총 {df.shape[0]}개의 항목, {df.shape[1]}개의 컬럼)")

    print("\n--- DataFrame 정보 (df.info()) ---")
    # .info()는 각 컬럼의 이름, null이 아닌 값의 개수, 데이터 타입을 보여줍니다.
    df.info()

    print("\n--- 상위 5개 항목 (df.head()) ---")
    # .head()로 실제 데이터가 어떻게 들어갔는지 샘플을 봅니다.
    pd.set_option('display.max_columns', None) # 모든 컬럼을 다 보여주기
    pd.set_option('display.width', 1000)      # 넓게 보기
    print(df.head())

    # rag_text 컬럼만 따로 볼 수도 있습니다.
    # print("\n--- 'rag_text' 컬럼 샘플 ---")
    # print(df.loc[0, 'rag_text']) # 첫 번째 항목의 rag_text
    
except FileNotFoundError:
    print(f"❌ 오류: 파일을 찾을 수 없습니다. 경로를 확인하세요: {file_path}")
except Exception as e:
    print(f"❌ 오류: 파일을 읽는 중 문제가 발생했습니다: {e}")

✅ './output/rag_data.jsonl' 파일을 DataFrame으로 성공적으로 로드했습니다.
Shape: (860, 21) (총 860개의 항목, 21개의 컬럼)

--- DataFrame 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   qid               860 non-null    object 
 1   imdb_id           860 non-null    object 
 2   tmdb_id           860 non-null    int64  
 3   type              860 non-null    object 
 4   title_ko          860 non-null    object 
 5   title_en          849 non-null    object 
 6   year              860 non-null    int64  
 7   overview          860 non-null    object 
 8   genres            860 non-null    object 
 9   poster_path       537 non-null    object 
 10  backdrop_path     530 non-null    object 
 11  runtime_min       540 non-null    float64
 12  cast              860 non-null    object 
 13  directors         860 non-null    object 
 14  writers

In [16]:
df

,qid,imdb_id,tmdb_id,type,title_ko,title_en,year,overview,genres,poster_path,backdrop_path,runtime_min,cast,directors,writers,ott_streaming_kr,ott_rent_kr,ott_buy_kr,keywords,rating_kr,rag_text
0,http://www.wikidata.org/entity/Q110393291,tt13498824,760497,movie,승부,The Match,2025,세계 최고 바둑 대회에서 국내 최초 우승자가 된 조훈현. 전 국민적 영웅으로 대접받...,[드라마],/x1xHVv5rdvvCjF5wmGhtbHrlUgo.jpg,/tVCg0HYPLFVYljiQXJCWTY43LJR.jpg,115.0,"[이병헌, 유아인, 고창석, 현봉식, 문정희, 조우진, 손종학, 김강훈, 남문철, ...",[김형주],"[김형주, 윤종빈]","[Netflix, Netflix Standard with Ads]",[],[],"[based on true story, go]",12,[제목] 승부\n[영문 제목] The Match\n[줄거리] 세계 최고 바둑 대회에...
1,http://www.wikidata.org/entity/Q110585119,tt14044924,793058,movie,하이파이브,Hi-Five,2025,"태권소녀 완서, 작가 지망생 지성, 후레쉬 매니저 선녀, FM 작업반장 약선 그리...","[SF, 액션, 코미디, 판타지]",/xpqa2ShXecFxMUGKv82Zx56ZoUD.jpg,/oBV1aajDou6hvPD55vQHrBSlDgb.jpg,120.0,"[이재인, 안재홍, 유아인, 라미란, 김희원, 오정세, 신구, 진영, 진희경, 장광]",[강형철],[강형철],[Disney Plus],[],[],"[super power, organ transplant, superhero team]",15,"[제목] 하이파이브\n[영문 제목] Hi-Five\n[줄거리] 태권소녀 완서, 작가..."
2,http://www.wikidata.org/entity/Q112977213,tt12299608,696506,movie,미키 17,Mickey 17,2025,친구 티모와 함께 차린 마카롱 가게가 쫄딱 망해 거액의 빚을 지고 못 갚으면 죽이겠...,"[SF, 코미디, 모험]",/mH7QnJDxQibVZw0M66IBZbsw2O6.jpg,/pkZ8SaIQwjBuS1fcJJE7Are7IrL.jpg,137.0,"[로버트 패틴슨, 나오미 애키, 스티븐 연, 마크 러팔로, 토니 콜렛, 아나마리아 ...",[봉준호],[봉준호],[],[Google Play Movies],[Google Play Movies],"[based on novel or book, dark comedy, space tr...",15,[제목] 미키 17\n[영문 제목] Mickey 17\n[줄거리] 친구 티모와 함께...
3,http://www.wikidata.org/entity/Q120922062,tt28488187,973628,movie,보통의 가족,A Normal Family,2025,물질적 욕망을 우선시하며 살인자의 변호도 마다하지 않는 변호사 재완과 원리원칙을 중...,"[드라마, 스릴러]",/chuNOF8p6Y1mJ0McFKa17ojXAsL.jpg,/xqdtTCl1xoLgWnzFCBQ0vv4iSmR.jpg,109.0,"[설경구, 장동건, 김희애, 수현, 홍예지, 김정철, 최리, 유수빈, 변중희, Ah...",[허진호],"[Park Eun-kyo, 박준석]",[Disney Plus],"[wavve, Google Play Movies]","[wavve, Google Play Movies]","[based on novel or book, dark comedy]",12,[제목] 보통의 가족\n[영문 제목] A Normal Family\n[줄거리] 물질...
4,http://www.wikidata.org/entity/Q125971387,tt12300742,701387,movie,부고니아,None,2025,"벌들은 사라지고, 지구는 병들고 있고, 인류는 고통받고 있다. 거대 바이오 기업의 ...","[SF, 범죄, 스릴러]",/hHCuWbwZvywOTWTk1CVIRMQgnkB.jpg,/jihTSrMksmU5ujqRZnc6SBFSyfq.jpg,119.0,"[엠마 스톤, 제시 플레먼스, 에이단 델비스, 스타브로스 할키어스, 알리시아 실버스...",[요르고스 란티모스],[윌 트레이시],[],[],[],"[saving the world, capitalism, kidnapping, par...",15,"[제목] 부고니아\n[영문 제목] nan\n[줄거리] 벌들은 사라지고, 지구는 병들..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,http://www.wikidata.org/entity/Q50322115,tt4722674,383353,movie,Naneun Seonmuda,I Am Sun Mu,2015,서울에서 김정일 체제에 대한 풍자 그림을 그리고 자신의 탈북으로 비롯된 아픔을 극복...,"[애니메이션, 다큐멘터리]",NaN,NaN,NaN,"[Sun Mu, Angeline, Cui Xianji, Liang Kegang, H...",[Adam Sjöberg],[Adam Sjöberg],[],NaN,NaN,"[biography, news, north korean defector, north...",,[제목] Naneun Seonmuda\n[영문 제목] I Am Sun Mu\n[줄거...
856,http://www.wikidata.org/entity/Q55734574,tt5258552,449802,movie,중급불어,None,2015,,[드라마],NaN,NaN,NaN,"[Yoon Geum-sun, Gregor Mc Ghee]",[Yann Kerloc'h],[Yann Kerloc'h],[],NaN,NaN,"[exam, viddsee, short film]",,[제목] 중급불어\n[영문 제목] nan\n[추가 요약] 《중급불어》는 2015년에...
857,http://www.wikidata.org/entity/Q65241097,tt7520904,601068,movie,다방의 푸른 꿈,Try To Remember,2015,전쟁의 폐허 속에서 음악에 대한 천부적인 재능을 통해 국내 최초 걸그룹 결성! 듣는...,"[다큐멘터리, 음악]",NaN,NaN,NaN,"[김민자, 김숙자, 김애자, Nan-yeong Lee, Kim Hae-song]",[Kim Dae-hyun],[Kim Dae-hyun],[],NaN,NaN,"[girl group, k-pop]",,[제목] 다방의 푸른 꿈\n[영문 제목] Try To Remember\n[줄거리] ...
858,http://www.wikidata.org/entity/Q65300188,tt5358748,510589,movie,퀴어영화 나비: 어른들의 일,Butterfly: The Adult World,2015,,[드라마],NaN,NaN,NaN,"[Daeyoung Kim, Yeongdeok Lim, Gwangmu Moon, Se...",[Inkyu Baek],[Inkyu Baek],"[wavve, TVING]",NaN,NaN,"[gay club, boys' love (bl)]",,[제목] 퀴어영화 나비: 어른들의 일\n[영문 제목] Butterfly: The A...
